CTGAN Model
===========

In this guide we will go through a series of steps that will let you
discover functionalities of the `CTGAN` model, including how to:

-   Create an instance of `CTGAN`.
-   Fit the instance to your data.
-   Generate synthetic versions of your data.
-   Use `CTGAN` to anonymize PII information.
-   Customize the data transformations to improve the learning process.
-   Specify hyperparameters to improve the output quality.

What is CTGAN?
--------------

The `sdv.tabular.CTGAN` model is based on the GAN-based Deep Learning
data synthesizer which was presented at the NeurIPS 2020 conference by
the paper titled [Modeling Tabular data using Conditional
GAN](https://arxiv.org/abs/1907.00503).

Let\'s now discover how to learn a dataset and later on generate
synthetic data with the same format and statistical properties by using
the `CTGAN` class from SDV.

Quick Usage
-----------

We will start by loading one of our demo datasets, the
`student_placements`, which contains information about MBA students that
applied for placements during the year 2020.

<div class="alert alert-warning">

**Warning**

In order to follow this guide you need to have `ctgan` installed on your
system. If you have not done it yet, please install `ctgan` now by
executing the command `pip install sdv` in a terminal.

</div>

In [1]:
from sdv.demo import load_tabular_demo

data = load_tabular_demo('student_placements')
data.head()

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,17264,M,67.00,91.00,Commerce,58.00,Sci&Tech,False,0,55.0,Mkt&HR,58.80,27000.0,True,2020-07-23,2020-10-12,3.0
1,17265,M,79.33,78.33,Science,77.48,Sci&Tech,True,1,86.5,Mkt&Fin,66.28,20000.0,True,2020-01-11,2020-04-09,3.0
2,17266,M,65.00,68.00,Arts,64.00,Comm&Mgmt,False,0,75.0,Mkt&Fin,57.80,25000.0,True,2020-01-26,2020-07-13,6.0
3,17267,M,56.00,52.00,Science,52.00,Sci&Tech,False,0,66.0,Mkt&HR,59.43,NaN,False,NaT,NaT,NaN
4,17268,M,85.80,73.60,Commerce,73.30,Comm&Mgmt,False,0,96.8,Mkt&Fin,55.50,42500.0,True,2020-07-04,2020-09-27,3.0


As you can see, this table contains information about students which
includes, among other things:

-   Their id and gender
-   Their grades and specializations
-   Their work experience
-   The salary that they where offered
-   The duration and dates of their placement

You will notice that there is data with the following characteristics:

-   There are float, integer, boolean, categorical and datetime values.
-   There are some variables that have missing data. In particular, all
    the data related to the placement details is missing in the rows
    where the student was not placed.

Let us use `CTGAN` to learn this data and then sample synthetic data
about new students to see how well de model captures the characteristics
indicated above. In order to do this you will need to:

-   Import the `sdv.tabular.CTGAN` class and create an instance of it.
-   Call its `fit` method passing our table.
-   Call its `sample` method indicating the number of synthetic rows
    that you want to generate.

In [2]:
from sdv.tabular import CTGAN

model = CTGAN()
model.fit(data)

/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_components=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_components=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_components=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_components=10 as keyword args. From versi

<div class="alert alert-info">

**Note**

Notice that the model `fitting` process took care of transforming the
different fields using the appropriate [Reversible Data
Transforms](http://github.com/sdv-dev/RDT) to ensure that the data has a
format that the underlying CTGANSynthesizer class can handle.

</div>

### Generate synthetic data from the model

Once the modeling has finished you are ready to generate new synthetic
data by calling the `sample` method from your model passing the number
of rows that we want to generate.

In [3]:
new_data = model.sample(200)

This will return a table identical to the one which the model was fitted
on, but filled with new data which resembles the original one.

In [4]:
new_data.head()

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,17457,M,47.292072,69.769094,Science,72.418410,Comm&Mgmt,True,0,82.974995,Mkt&Fin,65.289259,NaN,True,2020-01-02,2020-10-15,NaN
1,17443,M,46.460769,58.735737,Science,61.258294,Comm&Mgmt,False,0,103.723943,Mkt&HR,66.653649,32276.222436,False,NaT,2020-07-29,6.0
2,17412,M,84.966895,76.651983,Science,65.385269,Others,False,0,90.814337,Mkt&HR,58.472921,20729.956404,False,NaT,2020-03-25,12.0
3,17361,M,36.803023,53.451372,Commerce,57.771001,Comm&Mgmt,False,0,69.127622,Mkt&HR,66.056195,NaN,True,2019-12-23,2020-09-14,NaN
4,17301,M,56.480312,53.000355,Science,72.266995,Comm&Mgmt,False,0,79.023794,Mkt&HR,65.929123,30957.395420,False,NaT,2020-09-04,12.0


<div class="alert alert-info">

**Note**

You can control the number of rows by specifying the number of `samples`
in the `model.sample(<num_rows>)`. To test, try `model.sample(10000)`.
Note that the original table only had \~200 rows.

</div>

### Save and Load the model

In many scenarios it will be convenient to generate synthetic versions
of your data directly in systems that do not have access to the original
data source. For example, if you may want to generate testing data on
the fly inside a testing environment that does not have access to your
production database. In these scenarios, fitting the model with real
data every time that you need to generate new data is feasible, so you
will need to fit a model in your production environment, save the fitted
model into a file, send this file to the testing environment and then
load it there to be able to `sample` from it.

Let\'s see how this process works.

#### Save and share the model

Once you have fitted the model, all you need to do is call its `save`
method passing the name of the file in which you want to save the model.
Note that the extension of the filename is not relevant, but we will be
using the `.pkl` extension to highlight that the serialization protocol
used is [pickle](https://docs.python.org/3/library/pickle.html).

In [5]:
model.save('my_model.pkl')

This will have created a file called `my_model.pkl` in the same
directory in which you are running SDV.

<div class="alert alert-info">

**Important**

If you inspect the generated file you will notice that its size is much
smaller than the size of the data that you used to generate it. This is
because the serialized model contains **no information about the
original data**, other than the parameters it needs to generate
synthetic versions of it. This means that you can safely share this
`my_model.pkl` file without the risc of disclosing any of your real
data!

</div>

#### Load the model and generate new data

The file you just generated can be send over to the system where the
synthetic data will be generated. Once it is there, you can load it
using the `CTGAN.load` method, and then you are ready to sample new data
from the loaded instance:

In [6]:
loaded = CTGAN.load('my_model.pkl')
new_data = loaded.sample(200)

<div class="alert alert-warning">

**Warning**

Notice that the system where the model is loaded needs to also have
`sdv` and `ctgan` installed, otherwise it will not be able to load the
model and use it.

</div>

### Specifying the Primary Key of the table

One of the first things that you may have noticed when looking that demo
data is that there is a `student_id` column which acts as the primary
key of the table, and which is supposed to have unique values. Indeed,
if we look at the number of times that each value appears, we see that
all of them appear at most once:

In [7]:
data.student_id.value_counts().max()

1

However, if we look at the synthetic data that we generated, we observe
that there are some values that appear more than once:

In [8]:
new_data[new_data.student_id == new_data.student_id.value_counts().index[0]]

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
29,17384,M,91.934439,51.200693,Science,50.484399,Others,False,0,73.271414,Mkt&HR,63.632672,NaN,True,2020-03-24,2020-09-25,3.0
66,17384,F,31.951812,56.901614,Commerce,68.925051,Comm&Mgmt,False,0,75.089076,Mkt&Fin,64.322553,NaN,False,NaT,NaT,6.0
87,17384,F,47.829760,69.246069,Commerce,56.409420,Comm&Mgmt,False,0,65.373366,Mkt&Fin,74.053660,25857.217277,False,2020-01-02,2020-03-11,3.0
142,17384,M,75.481918,63.823412,Commerce,56.835852,Comm&Mgmt,True,1,64.846230,Mkt&Fin,63.631237,19926.874480,True,2019-12-25,2020-05-13,NaN
171,17384,F,64.275994,55.116643,Commerce,70.200575,Comm&Mgmt,False,0,58.042507,Mkt&Fin,71.791256,29670.550542,True,NaT,NaT,12.0
178,17384,F,78.905610,49.315453,Commerce,90.596275,Comm&Mgmt,False,1,69.059069,Mkt&Fin,75.427640,22556.859830,True,NaT,NaT,6.0


This happens because the model was not notified at any point about the
fact that the `student_id` had to be unique, so when it generates new
data it will provoke collisions sooner or later. In order to solve this,
we can pass the argument `primary_key` to our model when we create it,
indicating the name of the column that is the index of the table.

In [9]:
model = CTGAN(
    primary_key='student_id'
)
model.fit(data)
new_data = model.sample(200)
new_data.head()

/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_components=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/mixture/_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn('Initialization %d did not converge. '
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_components=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_components=10 as keyword args. From versi

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,0,F,74.632413,56.599017,Commerce,67.045563,Comm&Mgmt,False,0,57.442891,Mkt&Fin,57.444657,25943.646776,False,NaT,NaT,12.0
1,1,F,63.701896,60.543102,Science,74.860734,Comm&Mgmt,False,1,72.050981,Mkt&HR,57.679963,27299.461553,False,2020-08-13,NaT,NaN
2,2,F,42.894110,58.937303,Commerce,80.608819,Comm&Mgmt,False,0,76.251312,Mkt&Fin,57.933377,25056.936587,False,2020-04-03,2020-11-23,6.0
3,3,M,46.202521,57.395240,Commerce,67.678547,Comm&Mgmt,False,0,94.991826,Mkt&HR,49.672453,18093.116965,False,2020-03-18,2020-08-08,NaN
4,4,M,58.529786,40.357885,Commerce,71.650460,Sci&Tech,False,0,93.907926,Mkt&Fin,68.204984,20835.604758,True,NaT,NaT,3.0


As a result, the model will learn that this column must be unique and
generate a unique sequence of values for the column:

In [10]:
new_data.student_id.value_counts().max()

1

### Anonymizing Personally Identifiable Information (PII)

There will be many cases where the data will contain Personally
Identifiable Information which we cannot disclose. In these cases, we
will want our Tabular Models to replace the information within these
fields with fake, simulated data that looks similar to the real one but
does not contain any of the original values.

Let\'s load a new dataset that contains a PII field, the
`student_placements_pii` demo, and try to generate synthetic versions of
it that do not contain any of the PII fields.

<div class="alert alert-info">

**Note**

The `student_placements_pii` dataset is a modified version of the
`student_placements` dataset with one new field, `address`, which
contains PII information about the students. Notice that this additional
`address` field has been simulated and does not correspond to data from
the real users.

</div>

In [11]:
data_pii = load_tabular_demo('student_placements_pii')
data_pii.head()

,student_id,address,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,17264,"70304 Baker Turnpike\nEricborough, MS 15086",M,67.00,91.00,Commerce,58.00,Sci&Tech,False,0,55.0,Mkt&HR,58.80,27000.0,True,2020-07-23,2020-10-12,3.0
1,17265,"805 Herrera Avenue Apt. 134\nMaryview, NJ 36510",M,79.33,78.33,Science,77.48,Sci&Tech,True,1,86.5,Mkt&Fin,66.28,20000.0,True,2020-01-11,2020-04-09,3.0
2,17266,"3702 Bradley Island\nNorth Victor, FL 12268",M,65.00,68.00,Arts,64.00,Comm&Mgmt,False,0,75.0,Mkt&Fin,57.80,25000.0,True,2020-01-26,2020-07-13,6.0
3,17267,Unit 0879 Box 3878\nDPO AP 42663,M,56.00,52.00,Science,52.00,Sci&Tech,False,0,66.0,Mkt&HR,59.43,NaN,False,NaT,NaT,NaN
4,17268,"96493 Kelly Canyon Apt. 145\nEast Steven, NC 3...",M,85.80,73.60,Commerce,73.30,Comm&Mgmt,False,0,96.8,Mkt&Fin,55.50,42500.0,True,2020-07-04,2020-09-27,3.0


If we use our tabular model on this new data we will see how the
synthetic data that it generates discloses the addresses from the real
students:

In [12]:
model = CTGAN(
    primary_key='student_id',
)
model.fit(data_pii)
new_data_pii = model.sample(200)
new_data_pii.head()

/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_components=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_components=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/mixture/_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn('Initialization %d did not converge. '
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_components=10 as keyword args. From versi

,student_id,address,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,0,"048 Molly Wells Apt. 471\nEast Robertville, FL...",M,95.367454,73.573789,Commerce,80.317664,Comm&Mgmt,False,0,74.816614,Mkt&HR,57.915350,NaN,True,2020-01-01,2020-11-25,6.0
1,1,"926 Martinez Unions\nAllisonstad, AK 04702",M,67.702313,78.239946,Science,64.001437,Comm&Mgmt,False,0,100.098672,Mkt&HR,78.562662,30435.348524,True,2020-03-08,2020-12-13,12.0
2,2,"8034 Freeman Meadows\nSouth Bryce, NJ 14719",M,72.067470,67.631672,Commerce,49.159770,Others,True,0,92.139077,Mkt&HR,62.674147,NaN,True,NaT,2020-08-02,NaN
3,3,"2348 Washington Oval Apt. 978\nOwensbury, HI 7...",M,70.897796,37.843001,Science,55.837035,Comm&Mgmt,False,0,81.146561,Mkt&HR,65.288252,NaN,True,NaT,2020-06-21,3.0
4,4,2213 Brittney Springs Apt. 360\nLake Maliktown...,F,70.966723,38.326197,Commerce,55.809746,Comm&Mgmt,False,0,111.381936,Mkt&Fin,78.924485,NaN,False,2019-12-29,2020-07-18,6.0


More specifically, we can see how all the addresses that have been
generated actually come from the original dataset:

In [13]:
new_data_pii.address.isin(data_pii.address).sum()

200

In order to solve this, we can pass an additional argument
`anonymize_fields` to our model when we create the instance. This
`anonymize_fields` argument will need to be a dictionary that contains:

-   The name of the field that we want to anonymize.
-   The category of the field that we want to use when we generate fake
    values for it.

The list complete list of possible categories can be seen in the [Faker
Providers](https://faker.readthedocs.io/en/master/providers.html) page,
and it contains a huge list of concepts such as:

-   name
-   address
-   country
-   city
-   ssn
-   credit_card_number
-   credit_card_expire
-   credit_card_security_code
-   email
-   telephone
-   \...

In this case, since the field is an e-mail address, we will pass a
dictionary indicating the category `address`

In [14]:
model = CTGAN(
    primary_key='student_id',
    anonymize_fields={
        'address': 'address'
    }
)
model.fit(data_pii)

/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_components=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/mixture/_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn('Initialization %d did not converge. '
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_components=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/mixture/_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try differ

As a result, we can see how the real `address` values have been replaced
by other fake addresses:

In [15]:
new_data_pii = model.sample(200)
new_data_pii.head()

,student_id,address,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,0,"32736 Carpenter Estate\nEast Tammyland, HI 35089",M,72.867664,74.127458,Science,64.963277,Comm&Mgmt,True,0,44.573698,Mkt&Fin,61.686145,27761.042870,False,2020-03-28,2020-09-06,NaN
1,1,"94234 Lamb Drive Suite 837\nPort Mitchell, MS ...",M,75.864244,66.101474,Science,64.721835,Comm&Mgmt,False,0,68.308982,Mkt&HR,55.343451,27550.417239,True,2020-07-08,2020-07-22,NaN
2,2,"517 Joshua Route Apt. 543\nDestinyton, GA 01241",F,75.848736,75.924939,Science,75.994438,Comm&Mgmt,False,0,53.491070,Mkt&Fin,58.026820,20279.012468,True,2020-02-28,NaT,12.0
3,3,57539 Alexander Light Suite 282\nKimberlyshire...,F,76.226335,67.822484,Commerce,69.196293,Others,True,0,54.299480,Mkt&HR,51.279860,NaN,False,2020-07-23,NaT,6.0
4,4,"217 Wells Corner Apt. 919\nNew Brianna, MI 96887",F,56.012342,75.311186,Science,70.571361,Comm&Mgmt,False,1,100.664076,Mkt&Fin,47.568291,29539.496174,True,2020-01-30,2020-08-17,NaN


Which means that none of the original addresses can be found in the
sampled data:

In [16]:
data_pii.address.isin(new_data_pii.address).sum()

0

Advanced Usage
--------------

Now that we have discovered the basics, let\'s go over a few more
advanced usage examples and see the different arguments that we can pass
to our `CTGAN` Model in order to customize it to our needs.

### How to modify the CTGAN Hyperparameters?

A part from the common Tabular Model arguments, `CTGAN` has a number of
additional hyperparameters that control its learning behavior and can
impact on the performance of the model, both in terms of quality of the
generated data and computational time.

-   `epochs` and `batch_size`: these arguments control the number of
    iterations that the model will perform to optimize its parameters,
    as well as the number of samples used in each step. Its default
    values are `300` and `500` respectively, and `batch_size` needs to
    always be a value which is multiple of `10`. These hyperparameters
    have a very direct effect in time the training process lasts but
    also on the performance of the data, so for new datasets, you might
    want to start by setting a low value on both of them to see how long
    the training process takes on your data and later on increase the
    number to acceptable values in order to improve the performance.
-   `log_frequency`: Whether to use log frequency of categorical levels
    in conditional sampling. It defaults to `True`. This argument affects
    how the model processes the frequencies of the categorical values that
    are used to condition the rest of the values. In some cases, changing
    it to `False` could lead to better performance.
-   `embedding_dim` (int): Size of the random sample passed to the
    Generator. Defaults to 128.
-   `generator_dim` (tuple or list of ints): Size of the output samples for
    each one of the Residuals. A Resiudal Layer will be created for each
    one of the values provided. Defaults to (256, 256).
-   `discriminator_dim` (tuple or list of ints): Size of the output samples for
    each one of the Discriminator Layers. A Linear Layer will be created
    for each one of the values provided. Defaults to (256, 256).
-   `generator_lr` (float): Learning rate for the generator. Defaults to 2e-4.
-   `generator_decay` (float): Generator weight decay for the Adam Optimizer.
    Defaults to 1e-6.
-   `discriminator_lr` (float): Learning rate for the discriminator.
    Defaults to 2e-4.
-   `discriminator_decay` (float): Discriminator weight decay for the Adam
    Optimizer. Defaults to 1e-6.
-   `discriminator_steps` (int): Number of discriminator updates to do for
    each generator update. From the WGAN paper: https://arxiv.org/abs/1701.07875.
    WGAN paper default is 5. Default used is 1 to match original CTGAN
    implementation.
-   `verbose`: Whether to print fit progress on stdout. Defaults to
    `False`.

<div class="alert alert-warning">

**Warning**

Notice that the value that you set on the `batch_size` argument must
always be a multiple of `10`!

</div>

As an example, we will try to fit the `CTGAN` model slightly increasing
the number of epochs, reducing the `batch_size`, adding one additional
layer to the models involved and using a smaller wright decay.

Before we start, we will evaluate the quality of the previously
generated data using the `sdv.evaluation.evaluate` function

In [17]:
from sdv.evaluation import evaluate

evaluate(new_data, data)

/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sdmetrics/column_pairs/statistical/kl_divergence.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  real_data[pd.isnull(real_data)] = 0.0
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3086: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sdmetrics/column_pairs/statistical/kl_divergence.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
T

0.32406462093973115

Afterwards, we create a new instance of the `CTGAN` model with the
hyperparameter values that we want to use

In [18]:
model = CTGAN(
    primary_key='student_id',
    epochs=500,
    batch_size=100,
    generator_dim=(256, 256, 256),
    discriminator_dim=(256, 256, 256)
)

And fit to our data.

In [19]:
model.fit(data)

/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_components=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/mixture/_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn('Initialization %d did not converge. '
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_components=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/mixture/_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try differ

Finally, we are ready to generate new data and evaluate the results.

In [20]:
new_data = model.sample(len(data))
evaluate(new_data, data)

/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/mixture/_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn('Initialization %d did not converge. '
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sdmetrics/column_pairs/statistical/kl_divergence.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  real_data[pd.isnull(real_data)] = 0.0
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3086: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing

0.3159118904712087

As we can see, in this case these modifications changed the obtained
results slightly, but they did neither introduce dramatic changes in the
performance.


### Conditional Sampling

As the name implies, conditional sampling allows us to sample from a conditional distribution using the `CTGAN` model, which means we can generate only values that satisfy certain conditions. These conditional values can be passed to the `conditions` parameter in the `sample` method either as a dataframe or a dictionary.

In case a dictionary is passed, the model will generate as many rows as requested, all of which will satisfy the specified conditions, such as `gender = M`.

In [21]:
conditions = {
    "gender": 'M'
}
model.sample(5, conditions=conditions)

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,0,M,78.108272,63.031564,Commerce,56.250202,Comm&Mgmt,False,0,90.392488,Mkt&HR,60.983012,NaN,False,2020-03-31,2020-09-20,NaN
1,1,M,94.307154,55.415623,Commerce,72.065556,Sci&Tech,False,0,54.047342,Mkt&HR,55.655778,25970.504227,False,2019-12-22,2020-10-23,3.0
2,2,M,91.641509,30.424119,Science,70.310664,Sci&Tech,False,0,68.236851,Mkt&Fin,52.107014,28864.796343,True,2020-01-13,2020-04-24,3.0
3,3,M,90.955873,74.089762,Science,66.245198,Comm&Mgmt,True,0,104.551972,Mkt&HR,58.922370,30269.221118,True,2020-01-28,2020-11-09,3.0
4,0,M,66.763489,31.717517,Commerce,66.719464,Sci&Tech,True,0,54.598101,Mkt&Fin,64.016839,NaN,False,2020-05-10,2020-12-01,3.0


It's also possible to condition on multiple columns, such as `gender = M, work_experience = False, high_spec = Science`.

In [114]:
Scienceconditions = {
    'gender': 'M',
    'work_experience': False,
    'high_spec': 'Science'
}
model.sample(5, conditions=conditions)

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,1,M,79.265084,61.197548,Commerce,82.728840,Comm&Mgmt,False,0,87.061166,Mkt&Fin,83.296236,30853.507866,True,2020-01-14,2020-09-27,3.0
1,2,M,75.568486,26.792775,Commerce,68.991704,Comm&Mgmt,False,0,48.422110,Mkt&HR,51.238807,NaN,False,2020-03-01,2020-09-19,6.0
2,4,M,60.183634,62.750267,Commerce,68.373246,Comm&Mgmt,False,0,90.280342,Mkt&Fin,64.727951,NaN,False,2020-02-09,2021-01-21,NaN
3,0,M,69.579779,57.972240,Commerce,78.096314,Sci&Tech,False,0,99.731752,Mkt&HR,62.698823,NaN,False,2020-03-01,2020-06-07,12.0
4,0,M,71.237240,54.607124,Commerce,73.708724,Sci&Tech,False,0,66.852119,Mkt&HR,57.753672,26122.419754,False,2020-03-24,2021-02-25,12.0


`conditions` can also be passed as a dataframe. In that case, the model will generate one sample for each row of the dataframe, sorted in the same order. Since the model already knows how many samples to generate, passing it as a parameter is unnecessary. For example, if we want to generate three samples where `gender = M` and three samples with `gender = F`, we can do the following: 

In [115]:
import pandas as pd 

conditions = pd.DataFrame({
    'gender': ['M', 'M', 'M', 'F', 'F', 'F'],
    'work_experience': [True, True, True, True, True, True]
})
model.sample(conditions=conditions)

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,1,M,75.998917,47.639287,Commerce,75.532017,Sci&Tech,True,0,80.672773,Mkt&HR,63.343030,NaN,True,2020-03-05,2020-04-30,NaN
1,19,M,76.681427,49.933496,Science,71.340813,Sci&Tech,True,0,97.659748,Mkt&HR,60.699041,30564.659576,True,2020-06-08,2020-08-04,3.0
2,26,M,66.317708,43.259570,Commerce,79.278712,Sci&Tech,True,0,98.846265,Mkt&HR,50.988833,27846.914455,True,2020-01-29,2020-07-07,12.0
3,0,F,95.533489,63.755309,Science,84.582167,Comm&Mgmt,True,0,58.902333,Mkt&Fin,55.862083,27766.022479,False,2020-08-30,2020-04-07,3.0
4,1,F,64.136739,57.408101,Science,80.161135,Comm&Mgmt,True,0,76.244263,Mkt&HR,73.543054,NaN,False,NaT,2020-04-14,6.0
5,2,F,91.790124,56.544408,Commerce,66.014746,Comm&Mgmt,True,0,97.524105,Mkt&Fin,68.585577,26652.212102,True,2020-07-20,2020-03-13,3.0


`CTGAN` also supports conditioning on continuous values, as long as the values are within the range of seen numbers. For example, if all the values of the dataset are within 0 and 1, `CTGAN` will not be able to set this value to 1000.

In [107]:
conditions = {
    'degree_perc': 70.0
}
model.sample(5, conditions=conditions)

/Users/fealho/Desktop/SDV/sdv/tabular/base.py:163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampled[column] = value
/Users/fealho/Desktop/SDV/sdv/tabular/base.py:163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampled[column] = value
/Users/fealho/Desktop/SDV/sdv/tabular/base.py:163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,0,M,71.973201,35.284724,Commerce,70.0,Comm&Mgmt,False,0,83.981117,Mkt&HR,74.869319,34617.438812,False,2020-01-27,2020-05-01,3.0
1,1,M,77.565621,22.344136,Commerce,70.0,Sci&Tech,False,0,92.037496,Mkt&HR,53.879108,28611.827970,False,2020-01-21,2020-12-17,3.0
2,17,M,78.959496,52.016265,Science,70.0,Comm&Mgmt,False,0,63.030460,Mkt&Fin,68.532291,NaN,False,2019-12-14,2020-12-18,3.0
3,46,M,78.729349,60.963628,Commerce,70.0,Comm&Mgmt,True,1,84.420188,Mkt&HR,50.343941,NaN,False,2020-02-03,2020-04-27,6.0
4,14,F,71.213172,53.080303,Science,70.0,Sci&Tech,False,1,97.204266,Mkt&HR,63.268051,NaN,False,2020-03-19,NaT,3.0


<div class="alert alert-info">

**Note**
Currently, conditional sampling works through a rejection sampling process, where rows are sampled repeatedly until one that satisfies the conditions is found. In case you are running into a `Could not get enough valid rows within x trials` or simply wish to optimize the results, there are three parameters that can be fine-tuned: max_rows_multiplier, max_retries and float_rtol. More information about these parameters can be found in the API section.


</div>

### How do I specify constraints?

If you look closely at the data you may notice that some properties were
not completely captured by the model. For example, you may have seen
that sometimes the model produces an `experience_years` number greater
than `0` while also indicating that `work_experience` is `False`. These
type of properties are what we call `Constraints` and can also be
handled using `SDV`. For further details about them please visit the
[Handling Constraints](04_Handling_Constraints.ipynb) guide.

### Can I evaluate the Synthetic Data?

A very common question when someone starts using **SDV** to generate
synthetic data is: *\"How good is the data that I just generated?\"*

In order to answer this question, **SDV** has a collection of metrics
and tools that allow you to compare the *real* that you provided and the
*synthetic* data that you generated using **SDV** or any other tool.

You can read more about this in the [Evaluating Synthetic Data Generators](
05_Evaluating_Synthetic_Data_Generators.ipynb) guide.